In [1]:
from pathlib import Path

import scipy.signal as signal
import matplotlib.pyplot as plt
import scipy.io as io
import numpy as np

In [2]:
data_path = Path("../../../WA_shelf_FullData")

In [3]:
mat_files = sorted(list(data_path.glob("*.mat")))
len(mat_files)

2227

## Training Data Generation:

In [ ]:
# build the dictionary for the data 


# build ping dictionary
d_bg = {}
for file in mat_files[-500:]:
    ff = io.loadmat(file)
    date_string = str(file).split(".")[-2].split("/")[-1].split("_")[-1]
    signal = ff["DataSegment1"][:,-1]  # background channel 4 (deepest channel)
    d_bg[date_string] = signal

: 

In [ ]:
# build the dictionary for the data 


# build ping dictionary
d_tr = {}
for file in mat_files[-500:]:
    ff = io.loadmat(file)
    date_string = str(file).split(".")[-2].split("/")[-1].split("_")[-1]
    signal = ff["DataSegment2"][:,-1]  #  transmission channel 4 (deepest channel)
    d_tr[date_string] = signal

In [ ]:
import pandas as pd
df_tr = pd.DataFrame(d_tr)

In [ ]:
df_bg = pd.DataFrame(d_bg)

In [ ]:
df_tr.to_csv("df_tr.csv", index=None)

In [ ]:
df_bg.to_csv("df_bg.csv", index=None)

## Testing Data Generation:

In [4]:
# build the dictionary for the data 


# build ping dictionary
d_bg = {}
for file in mat_files[-900:-500]:
    ff = io.loadmat(file)
    date_string = str(file).split(".")[-2].split("/")[-1].split("_")[-1]
    signal = ff["DataSegment1"][:,-1]  # background channel 4 (deepest channel)
    d_bg[date_string] = signal

In [5]:
# build the dictionary for the data 


# build ping dictionary
d_tr = {}
for file in mat_files[-900:-500]:
    ff = io.loadmat(file)
    date_string = str(file).split(".")[-2].split("/")[-1].split("_")[-1]
    signal = ff["DataSegment2"][:,-1]  # transmission channel 4 (deepest channel)
    d_tr[date_string] = signal

In [6]:
import pandas as pd
df_tr = pd.DataFrame(d_tr)

In [7]:
df_bg = pd.DataFrame(d_bg)

In [8]:
df_tr.to_csv("df_tr_test.csv", index=None)

In [9]:
df_bg.to_csv("df_bg_test.csv", index=None)

In [10]:
import scipy as sp

def BandPass(inputSignal, bandLimits, freqSample):
    sos = sp.signal.butter(4, bandLimits, 'bandpass', fs = freqSample, output='sos')
    outputSignal = sp.signal.sosfilt(sos,inputSignal)
    
    return outputSignal

In [11]:
flimits = [3450, 3550]

In [12]:
fs = 48000

In [13]:
bg_all, tr_all = {}, {}

for col in df_bg:

    bg_ch4_filt = BandPass(df_bg[col], flimits, fs)
    bg_all[col] = 20*np.log10(np.sqrt(np.mean(np.square(bg_ch4_filt.astype('float')))))


for col in df_tr:

    tr_ch4_filt = BandPass(df_tr[col], flimits, fs)
    tr_all[col] = 20*np.log10(np.sqrt(np.mean(np.square(tr_ch4_filt.astype('float')))))

In [14]:
# convert to DataFrame
df_bg_all = pd.Series(bg_all)
df_tr_all = pd.Series(tr_all)

In [15]:
# sort values
df_bg_all = df_bg_all.sort_values()
df_tr_all = df_tr_all.sort_values()

In [ ]:
# sort original table by sorted RMS
df_tr[df_tr_all.index].to_csv('df_tr_test_sorted.csv', index=None)
df_bg[df_bg_all.index].to_csv('df_bg_test_sorted.csv', index=None)

: 